In [104]:
import pandas as pd
import numpy as np
import json
import requests

import time
import re
from tqdm import tqdm

# Data Preparation for Small Data Project

## Get features 不用跑这个了现在

In [ ]:
import requests

API_KEY = "sk-np-auPB1C0B6W2uh8UqKUH3y3sDzShvbT98iYqmiF7eyjI0"
url = "https://www.neuronpedia.org/api/explanation/export"
querystring = {"modelId":"gpt2-small","saeId":"8-res_fs768-jb"}
headers = {"X-Api-Key": API_KEY}

response = requests.get(url, headers=headers, params=querystring)

In [ ]:
import pandas as pd
df = pd.DataFrame.from_dict(response.json())
df.head()

,modelId,layer,index,description,explanationModelName,typeName
0,gpt2-small,8-res_fs768-jb,82,ages of children,gpt-3.5-turbo,oai_token-act-pair
1,gpt2-small,8-res_fs768-jb,489,words related to specific locations,gpt-3.5-turbo,oai_token-act-pair
2,gpt2-small,8-res_fs768-jb,142,names of individuals,gpt-3.5-turbo,oai_token-act-pair
3,gpt2-small,8-res_fs768-jb,546,mentions of the color red,gpt-3.5-turbo,oai_token-act-pair
4,gpt2-small,8-res_fs768-jb,0,phrases related to international relations or ...,gpt-3.5-turbo,oai_token-act-pair


In [ ]:
df.to_csv("/content/drive/MyDrive/small-data-files/features.csv")

## Get LLM

In [40]:
from unsloth import FastLanguageModel
#model_name = "unsloth/Llama-3.2-3B-Instruct"
model_name = "unsloth/Meta-Llama-3.1-8B-Instruct"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    load_in_4bit = True,
)
print(model.config.max_position_embeddings)

==((====))==  Unsloth 2024.11.7: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: NVIDIA RTX 6000 Ada Generation. Max memory: 47.485 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 8.9. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
131072


In [41]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaExtendedRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): L

## Get Corpse File

In [14]:
import json
with open("corpse.json", "r") as json_file:
    corpse = json.load(json_file)
print(corpse[0]['text'])

I think it should be fixed on either UTC standard or UTC+1 year around, with the current zone offsets. 
 Moving timescales add a lot of complexity to the implementation of timekeeping systems and have [dubious value]( 
 I think seasonal shifting time made sense in the pre-electric past, when timekeeping was more flexible and artificial light was inefficient and often dangerous. 
 Now we have machines that work easily with simple timekeeping rules, and it's more beneficial to spend a small amount on energy for lighting, and save the larger cost of engineering things to work with the complex timekeeping rules, as well as saving the irritation to humans. 
 Lighting has gotten much more efficient over time; we can squeeze out a lot more photons per unit of energy from a 2012 CFL or LED than a candle could in 1780, or a lightbulb could in 1950. 
 There's a lot of room for improvement in how we use lights as well; as lighting control gets more intelligent, there will be a lot of savings from

## Inference

In [10]:
df = pd.read_csv("features.csv")
descriptions = ", ".join(df["description"].apply(lambda x:"\'"+x+"\'").tolist())

In [ ]:

# system_message = f'''
# You are a machine learning algorithm that returns a json that predict how features are presented in the user's the input sentence, evaluated using a value between -1 to 1.\n\
# 1 indicates that the sentence fully represents the feature, 0 indicates that the sentence doesn't represent the feature at all, and -1 indicates that the sentence fully represents the opposite of the feature.\n\
# From the following list of the feature, please first identify features, at most 20 features, that are most relevant to the input sentence (has high absolute value of the predicted value), and then predict the value of those features presented in the input sentence.\n\
# Please predict to the second decimal place. Please output a json to represent the prediction. Please do not output anything else. \n\
# The following is the list of features, separated by comma: [{descriptions}]
# '''


In [ ]:
system_message = f'''
You are a machine learning algorithm that must select no more than 20 features from the list below and return a dictionary that predict a score that measures how these features are presented in the user's input text
The score ranges between -1 to 1.1 indicates that the sentence fully represents the feature, 0 indicates that the sentence doesn't represent the feature at all, and -1 indicates that the sentence fully represents the opposite of the feature.\n\

Instructions:
Select up to 20 features from the following list. If fewer than 20 are highly relevant, select only those. Please ensure the selection is limited to 20 or fewer features.
Predict the score of those features presented in the input sentence.\n\
Please predict to the second decimal place. Please output a dictionary to represent the prediction. Please do not output anything else other than the dictionary. \n\
Only output the dictionary in the following format, with up to 20 entries:
{{'ages of children': 0.98, "feature_name2": -0.32, ... }}

The following is the list of features, separated by comma: [{descriptions}]
'''

In [95]:
# helper methods
def extract_answer(answer:str):
  answer = answer.split("<|eot_id|><|start_header_id|>assistant<|end_header_id|>")[1].replace("<|eot_id|>","").replace("\n","").strip()
  #answer = answer.replace('"', '\\"') # handle " in json string
  return answer


def validate_json_string(s):
  if s.endswith("}"):
    return s
  last_comma_index = s.rfind(",")
  if last_comma_index != -1:
    s = s[:last_comma_index]
    s += "}"
    return s
  else:
    return "invalid string"

def clean_input_text(text):
    text = re.split(r'tl;dr', text, flags=re.IGNORECASE)[0]
    text = re.split(r'tldr', text, flags=re.IGNORECASE)[0]
    text = re.sub(r"[\\'\"]", "", text)
    return text


In [105]:
#@title Here!!!!
input_text = clean_input_text(corpse[11]['text'])

feature_selection_prompt = '''The following is a list of features, separated by commas: [%s]
Select 10 features or less that are highly relevent to the user input text from list provided. Do not select features that are irrelevent. Do not select any features that are not on the list. 
Please output your answer in the following python dictionary format, order them by relevancy:
{"order 1":{"feature name":feature name from the list,"reason":reason for selection, no more than 1 sentence},"order 2":{"feature name":feature name from the list,"reason":reason for selection, no more than 1 sentence},...}
Please do not output anything else other than the dictionary.'''%descriptions

# feature_selection_prompt = f'''
# The following is a list of features, separated by commas: [{descriptions}]

# From this list of feature, select 10 features or less that are relevent to the user input text from the following list. Do not select features that are irrelevent. Do not select any features that are not on the list.
# Please output your selection in the following python list format ordered by relevance :["feature name 1","feature name 2",...]
# Please do not output anything else other than the list.


# '''


# feature_selection_prompt = f'''
# The following is a list of features, separated by commas: [{descriptions}]
# From this list, select 10 features or fewer that are highly relevant to the user input text. Do **not** select any features that are not on the list. Please only choose from the provided features.
# Please output your answer in the following Python dictionary format:
# {{"feature name 1": "reason 1",
#  "feature name 2": "reason 2",
#  ...
# }}
# The reason for each selection should be concise and no more than one sentence.
# If no features are relevant, output an empty dictionary {{}}.
# Please do not output anything else other than the dictionary.
# Please ensure that you **do not generate new features** or mention features not found in the list.
# '''

messages = [
    {
        "role": "system",
        "content": feature_selection_prompt ,
    },
    {"role": "user", "content": f"text: {input_text}"},
]
inputs = tokenizer.apply_chat_template(messages, tokenize = True, add_generation_prompt = True, return_tensors = "pt").to("cuda")
generated_content = model.generate(input_ids = inputs, max_new_tokens = 1024, use_cache = True,top_p=0.95,temperature=0.1)
decoded_text = tokenizer.decode(generated_content[0])
entry = validate_json_string((extract_answer(decoded_text)))
if entry != "invalid string":
  entry = json.loads(entry)
else:
  raise ValueError("invalid string")


# validate the features are in the dictionary
# feature_names = [entry[key]["feature name"] for key in entry.keys() if entry[key]["feature name"] in df["description"].to_list()]
# feature_names = [i for i in list(entry.keys()) if i in df["description"].to_list()]
feature_names = [entry[f'order {i}']['feature name'] for i in range(1,len(entry)) if entry[f'order {i}']['feature name'] in df['description'].to_list()]
assert len(feature_names) != 0

# run a second round of inference to get the score
generate_score_prompt= f'''
You are a machine learning algorithm that returns a dictionary that predict how features are presented in the user's the input text, evaluated using a score between -1 to 1.
1 indicates that the sentence fully represents the feature, 0 indicates that the sentence doesn't represent the feature at all, and -1 indicates that the sentence fully represents the opposite of the feature.\n\

Instructions:
From the following list of the feature, predict scores that meansure each features presented in the input sentence.
Please predict to the second decimal place. Please output a dictionary to represent the prediction. Please do not output anything else other than the dictionary. \n\
The output format should be:{{"feature name 1":value,"feature name 2":value,...}} \n\

The following is the list of features, separated by comma: [{",".join(feature_names)}]
'''

# second stage inference
messages = [
    {
        "role": "system",
        "content": generate_score_prompt ,
    },
    {"role": "user", "content": f"text: {input_text}"},
]
inputs = tokenizer.apply_chat_template(messages, tokenize = True, add_generation_prompt = True, return_tensors = "pt").to("cuda")
generated_content = model.generate(input_ids = inputs, max_new_tokens = 1024, use_cache = True,top_p=0.95,temperature=0.1)
decoded_text = tokenizer.decode(generated_content[0])
entry = validate_json_string((extract_answer(decoded_text)))
if entry != "invalid string":
  entry = json.loads(entry)
else:
  raise ValueError("invalid string")

output_entry ={}
output_features = []
output_scores =[]
output_feature_ids = []
# ensure all features are in the dictionary
for index,feature in enumerate(entry.keys()):
  if feature in df["description"].to_list(): # it exist in the dictionary
    output_features.append(feature)
    output_scores.append(entry[feature])
    output_feature_ids.append(np.where(feature ==df["description"].to_numpy())[0][0])

if len(output_features):
  output_entry ={
    "text":input_text,
    "features":output_features,
    "feature_ids":output_feature_ids,
    "scores":output_scores
  }


print(output_entry)


KeyboardInterrupt: 

### Main Loop

In [108]:
synth_data = []
for i in tqdm(range(len(corpse))):
    input_text = clean_input_text(corpse[i]['text'])

    # first round of inference: select prompt
    feature_selection_prompt = '''The following is a list of features, separated by commas: [%s]
    Select 10 features or less that are highly relevent to the user input text from list provided. Do not select features that are irrelevent. Do not select any features that are not on the list. 
    Please output your answer in the following python dictionary format, order them by relevancy:
    {"order 1":{"feature name":feature name from the list,"reason":reason for selection, no more than 1 sentence},"order 2":{"feature name":feature name from the list,"reason":reason for selection, no more than 1 sentence},...}
    Please do not output anything else other than the dictionary.'''%descriptions

    messages = [
    {"role": "system","content": feature_selection_prompt},
    {"role": "user", "content": f"text: {input_text}"},
    ]
    inputs = tokenizer.apply_chat_template(messages, tokenize = True, add_generation_prompt = True, return_tensors = "pt").to("cuda")
    generated_content = model.generate(input_ids = inputs, max_new_tokens = 1024, use_cache = True,top_p=0.95,temperature=0.1)
    decoded_text = tokenizer.decode(generated_content[0])
    entry = validate_json_string((extract_answer(decoded_text)))
    try:
        entry = json.loads(entry)
    except json.JSONDecodeError as e:
        continue # skip this entry if the json is not valid

    
    feature_names = [entry[f'order {i}']['feature name'] for i in range(1,len(entry)) if entry[f'order {i}']['feature name'] in df['description'].to_list()]
    if len(feature_names) ==0:
        continue # skip this entry if the features is not in my dictionary


    # second inference feed for scoring
    enerate_score_prompt= f'''
    You are a machine learning algorithm that returns a dictionary that predict how features are presented in the user's the input text, evaluated using a score between -1 to 1.
    1 indicates that the sentence fully represents the feature, 0 indicates that the sentence doesn't represent the feature at all, and -1 indicates that the sentence fully represents the opposite of the feature.\n\
    
    Instructions:
    From the following list of the feature, predict scores that meansure each features presented in the input sentence.
    Please predict to the second decimal place. Please output a dictionary to represent the prediction. Please do not output anything else other than the dictionary. \n\
    The output format should be:{{"feature name 1":value,"feature name 2":value,...}} \n\
    
    The following is the list of features, separated by comma: [{",".join(feature_names)}]
    '''
    messages = [
        {
            "role": "system",
            "content": generate_score_prompt ,
        },
        {"role": "user", "content": f"text: {input_text}"},
    ]
    inputs = tokenizer.apply_chat_template(messages, tokenize = True, add_generation_prompt = True, return_tensors = "pt").to("cuda")
    generated_content = model.generate(input_ids = inputs, max_new_tokens = 1024, use_cache = True,top_p=0.95,temperature=0.1)
    decoded_text = tokenizer.decode(generated_content[0])
    entry = validate_json_string((extract_answer(decoded_text)))
    try:
        entry = json.loads(entry)
    except json.JSONDecodeError as e:
        continue # skip this entry if the json is not valid

    # organize output
    output_entry ={}
    output_features = []
    output_scores =[]
    output_feature_ids = []
    # ensure all features are in the dictionary
    for index,feature in enumerate(entry.keys()):
      if feature in df["description"].to_list(): # it exist in the dictionary
        output_features.append(feature)
        output_scores.append(entry[feature])
        output_feature_ids.append(int(np.where(feature ==df["description"].to_numpy())[0][0]))
    
    if len(output_features):
      output_entry ={
        "text":input_text,
        "features":output_features,
        "feature_ids":output_feature_ids,
        "scores":output_scores
      }

    synth_data.append(output_entry)

    # write to the file for every 5 entries
    if len(synth_data)%5 ==0:
        with open("synth_data.txt", "a") as json_file:
            for entry in synth_data:
                json.dumps(entry)
                json_file.write("\n\n")
        synth_data=[]


    

  0%|          | 0/50000 [00:03<?, ?it/s]


KeyboardInterrupt: 

In [100]:
d = json.load([{"a":[1,2]},{"b":[1,2]}])
#d.update([{"c":[3,3]},{"d":[5,5]}])

AttributeError: 'list' object has no attribute 'read'

In [ ]:
#@title main loop
synth_data = []
for i in range(0,len(corpse)):
  # preprocess input text
  input_text = re.split(r'tl;dr', corpse[i]['text'], flags=re.IGNORECASE)[0] # only use what is before the tl;dr

  # feed into the model
  messages = [
      {
          "role": "system",
          "content": system_message,
      },
      {"role": "user", "content": f"sentence: {input_text}"},
  ]
  inputs = tokenizer.apply_chat_template(messages, tokenize = True, add_generation_prompt = True, return_tensors = "pt").to("cuda")
  generated_content = model.generate(input_ids = inputs, max_new_tokens = 1024, use_cache = True)


  decoded_text = tokenizer.decode(generated_content[0])

  # clean the output
  entry = extract_answer(decoded_text)

  # ensure all features are in the dictionary
  for index,feature in enumerate(entry.keys()):
    # search if the feature exist in the feature dictionary
    if feature in df["description"].to_list(): # it exist in the dictionary
      entry['features'][index]["feature id"] = np.argmax(ls)
    else: # it does not exist, remove the entry
      del entry['features'][index]

  entry['text'] = input_text
  synth_data.append(entry)


  # write to the file for every 10 entries
  if len(synth_data)%10 ==0:
    with open("/content/drive/MyDrive/small-data-files/synth_data.json", "r") as json_file:
      data = json.load(json_file)
    data.extend(synth_data)
    with open("/content/drive/MyDrive/small-data-files/synth_data.json", "w") as json_file:
      json.dump(data, json_file)

    synth_data = []

  0%|          | 0/5000 [01:05<?, ?it/s]


KeyError: 'features'

In [ ]:
#@title One sentence testing
input_text = re.split(r'tl;dr', corpse[0]['text'], flags=re.IGNORECASE)[0] # only use what is before the tl;dr

# feed into the model
messages = [
    {
        "role": "system",
        "content": system_message,
    },
    {"role": "user", "content": f"sentence: {input_text}"},
]
inputs = tokenizer.apply_chat_template(messages, tokenize = True, add_generation_prompt = True, return_tensors = "pt").to("cuda")
generated_content = model.generate(input_ids = inputs, max_new_tokens = 1024, use_cache = True,top_p=0.95,temperature=0.1)
decoded_text = tokenizer.decode(generated_content[0])


# clean the output
entry = validate_json_string(extract_answer(decoded_text))
if entry != "invalid string":
  entry = json.loads(entry)
else:
  raise ValueError("invalid string")

output_feature_ids = []
output_features = []
output_scores =[]
output_entry ={}
# ensure all features are in the dictionary
for index,feature in enumerate(entry.keys()):
  if feature in df["description"].to_list(): # it exist in the dictionary
    output_features.append(feature)
    output_scores.append(entry[feature])
    output_feature_ids.append(np.where(feature ==df["description"].to_numpy())[0][0])

if len(output_features):
  output_entry ={
    "text":input_text,
    "features":output_features,
    "feature_ids":output_feature_ids,
    "scores":output_scores
  }

  #synth_data.append(output_entry)
print(output_entry)

{}


{'ages_of_children': 0.22,
 'words_related_to_specific_locations': -0.15,
 'names_of_individuals': -0.37,
 'mentions_of_the_color_red': 0.11,
 'phrases_related_to_international_relations_or_policies': -0.21,
 'requests_for_instruction_or_help': -0.32,
 'contractions_involving_dont': -0.34,
 'locations_or_geographic_regions': -0.26,
 'mentions_of_car_accidents': -0.19,
 'terms_related_to_law_enforcement_and_legal_actions': -0.29,
 'information_related_to_governmental_policies_and_international_affairs': -0.33,
 'actions_related_to_implementing_bans_and_restrictions': -0.41,
 'information_related_to_historical_figures_and_events': -0.31,
 'phrases_related_to_showcasing_or_demonstrating_something': -0.38,
 'phrases_related_to_laws_regulations_policies_and_societal_issues': -0.42,
 'proper_nouns_related_to_various_topics': -0.28,
 'actions_or_abilities_described_in_a_positive_light': -0.45,
 'financial_terms_related_to_investments_and_trade': -0.39,
 'names_of_sports_teams_and_players': -0

In [ ]:
df["description"].to_list()[3]

'mentions of the color red'